<a href="https://colab.research.google.com/github/jklm23/zuheyouhua/blob/main/%E7%BB%84%E5%90%88%E4%BC%98%E5%8C%96%E5%AE%9E%E9%AA%8C%E4%B8%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

使用IMDB电影评论数据集进行正面/负面情绪分类。

In [9]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [10]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  '''
  绘制图像
  '''
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [11]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

准备训练数据

In [12]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [13]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
#使用序列0填充
train_dataset = train_dataset.padded_batch(BATCH_SIZE)  
test_dataset = test_dataset.padded_batch(BATCH_SIZE)

In [14]:
# 编码器
encoder = info.features['text'].encoder

构建训练模型

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

编译模型，配置训练过程：

In [16]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

模型训练

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset, 
                    validation_steps=30)

Epoch 1/10
130/391 [========>.....................] - ETA: 9:17 - loss: 0.6931 - accuracy: 0.5001

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))